In [5]:
import pandas as pd
import numpy as np
import sys
import os
from pathlib import Path
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Proj
import matplotlib.pyplot as plt

In [6]:
input_path = 'E:\\forked_canada_udp/data/inputs/'
output_path ='E:\\forked_canada_udp/data/outputs/typologies/'
city_name = "Toronto"
all_typology = pd.read_csv(output_path + str.lower(city_name)+'_final_output.csv', index_col = 0)

### Typologies

In [13]:
# by census tract
in_city = all_typology[all_typology["Region.Name"] == "Toronto"]
not_city = all_typology[all_typology["Region.Name"] != "Toronto"]

In [16]:
in_city.groupby("Typology").size().sum()

555

In [17]:
not_city.groupby("Typology").size().sum()

521

In [65]:
# proportion of census tracts according to typology
proportions = (all_typology.groupby(["Typology"]).size() / all_typology.groupby(["Typology"]).size().sum()).sort_values(ascending = False)
proportions

Typology
Stable Moderate/Mixed Income              0.393868
At Risk of Becoming Exclusive             0.250000
Low-Income/Susceptible to Displacement    0.110849
Stable/Advanced Exclusive                 0.084906
Becoming Exclusive                        0.056604
Advanced Gentrification                   0.047170
At Risk of Gentrification                 0.018868
Early/Ongoing Gentrification              0.014151
Super Gentrification and Exclusion        0.014151
Ongoing Displacement                      0.009434
dtype: float64

### Findings: all households

In [18]:
# by % of all households
(in_city.groupby("Typology").agg('sum')["hh_16"] / in_city["hh_16"].sum()).sort_values(ascending = False)

Typology
Stable Moderate/Mixed Income              0.272122
At Risk of Becoming Exclusive             0.222743
Becoming Exclusive                        0.168695
Low-Income/Susceptible to Displacement    0.128509
Advanced Gentrification                   0.087471
Early/Ongoing Gentrification              0.038583
Ongoing Displacement                      0.023066
Super Gentrification and Exclusion        0.019431
Stable/Advanced Exclusive                 0.014590
At Risk of Gentrification                 0.014071
Unavailable or Unreliable Data            0.010718
Name: hh_16, dtype: float64

### Findings: low-income households

In [19]:
# total number of low income households across the Vancouver CMA
li_hh_tot = in_city["all_li_count_16"].sum()
li_hh_tot

456431.7311617312

In [20]:
li_hh_tot / in_city["hh_16"].sum()

0.41228060154254054

In [21]:
(in_city.groupby("Typology").agg('sum')["all_li_count_16"] / li_hh_tot).sort_values(ascending = False)

Typology
Stable Moderate/Mixed Income              0.245900
At Risk of Becoming Exclusive             0.203299
Low-Income/Susceptible to Displacement    0.174750
Becoming Exclusive                        0.147049
Advanced Gentrification                   0.100467
Early/Ongoing Gentrification              0.052754
Ongoing Displacement                      0.032293
At Risk of Gentrification                 0.022420
Stable/Advanced Exclusive                 0.008331
Super Gentrification and Exclusion        0.007949
Unavailable or Unreliable Data            0.004790
Name: all_li_count_16, dtype: float64

In [71]:
# census tracts classified as EOG / ARG / LISD / OD
ARG_EOG_LISD_OD = (all_typology["Typology"] == "Early/Ongoing Gentrification") | (all_typology["Typology"] == "At Risk of Gentrification") | (all_typology["Typology"] == "Low-Income/Susceptible to Displacement") | (all_typology["Typology"] == "Ongoing Displacement") 

In [72]:
# total number of low income households living in an ARG or EOG census tract
li_ARG_EOG_LISD_OD = all_typology[ARG_EOG_LISD_OD]["all_li_count_16"].sum()
li_ARG_EOG_LISD_OD

100026.42228142229

In [73]:
# percentage of low income households living in an ARG or EOG tract
pct_li_ARG_or_EOG = li_ARG_EOG_LISD_OD / li_hh_tot
pct_li_ARG_or_EOG

0.27125717773346625

In [27]:
# geographic distribution of low income households in EOG or ARG tracts
all_typology[ARG_EOG_LISD_OD == 1].groupby("Region.Name").agg('sum')["all_li_count_16"].sort_values(ascending = False) / li_ARG_EOG_LISD_OD

Region.Name
Vancouver              0.368164
Richmond               0.135369
Surrey                 0.105596
Burnaby                0.096362
Coquitlam              0.069099
Langley                0.050581
Greater Vancouver A    0.034483
Maple Ridge            0.034106
North Vancouver        0.030629
White Rock             0.026995
New Westminster        0.026298
Capilano 5             0.022317
Name: all_li_count_16, dtype: float64

In [28]:
# distribution of low income households according to typology
(all_typology.groupby("Typology").agg('sum')["all_li_count_16"].sort_values(ascending = False) / li_hh_tot).sort_values(ascending = False)

Typology
Stable Moderate/Mixed Income              0.366411
At Risk of Becoming Exclusive             0.209315
Low-Income/Susceptible to Displacement    0.201664
Advanced Gentrification                   0.062189
Becoming Exclusive                        0.046594
Stable/Advanced Exclusive                 0.038505
At Risk of Gentrification                 0.027675
Early/Ongoing Gentrification              0.023830
Ongoing Displacement                      0.018088
Super Gentrification and Exclusion        0.005729
Name: all_li_count_16, dtype: float64

In [31]:
gentrification = all_typology[(all_typology["Typology"] == "Advanced Gentrification") |
                             (all_typology["Typology"] == "At Risk of Gentrification") |
                             (all_typology["Typology"] == "Early/Ongoing Gentrification")]

In [32]:
(gentrification.groupby("Region.Name").agg('sum')["hh_16"] / gentrification["hh_16"].sum()).sort_values(ascending = False)

Region.Name
Vancouver              0.298109
Burnaby                0.226935
New Westminster        0.171929
Surrey                 0.147239
Port Moody             0.054062
Maple Ridge            0.029949
Greater Vancouver A    0.027903
White Rock             0.023694
Richmond               0.020180
Name: hh_16, dtype: float64

In [41]:
# households living in tracts classified as experiencing or at risk of gentrification
(gentrification["hh_16"].sum() / all_typology["hh_16"].sum())

0.08465678944920471

In [34]:
exclusive = all_typology[(all_typology["Typology"] == "Becoming Exclusive") |
                             (all_typology["Typology"] == "At Risk of Becoming Exclusive") |
                             (all_typology["Typology"] == "Stable/Advanced Exclusive") |
                             (all_typology["Typology"] == "Super Gentrification and Exclusion")
                        ]

In [42]:
# households living in tracts classified as at risk of becoming or already exclusive to low income households
(exclusive["hh_16"].sum() / all_typology["hh_16"].sum())

0.3773523329348257

In [44]:
# geographic distribution of households living in tracts classified as at risk of becoming or already exclusive to low income households
(exclusive.groupby("Region.Name").agg('sum')["hh_16"] / exclusive["hh_16"].sum()).sort_values(ascending = False)

Region.Name
Surrey             0.283372
Vancouver          0.175901
Langley            0.084064
Delta              0.064411
Port Coquitlam     0.063990
Burnaby            0.060496
North Vancouver    0.059019
Coquitlam          0.052206
Maple Ridge        0.048214
Richmond           0.037562
Port Moody         0.025822
New Westminster    0.016638
Pitt Meadows       0.010655
White Rock         0.009740
West Vancouver     0.007910
Name: hh_16, dtype: float64